In [1]:
!pip install sacrebleu
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.7 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
# OPUS100 dataset (English ↔ Urdu)
dataset = load_dataset("opus100", "en-ur")
print(dataset)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/301k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/148M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/296k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/753913 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
    train: Dataset({
        features: ['translation'],
        num_rows: 753913
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
})


In [3]:
dataset["train"][0]

{'translation': {'en': 'Yet, remember, as We communed with Moses for forty nights you took the calf in his absence (and worshipped it), and you did wrong.',
  'ur': 'اورجب ہم نے موسیٰ سے چالیس رات کا وعدہ کیا پھر اس کے بعد تم نے بچھڑا بنا لیا حالانکہ تم ظالم تھے'}}

In [4]:
from transformers import pipeline
model_checkpoint = "Helsinki-NLP/opus-mt-en-ur"
translator = pipeline("translation", model=model_checkpoint, src_lang="en_XX", tgt_lang="ur_PK")
translator("you did wrong.")

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/306M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/306M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/816k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/848k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cuda:0


[{'translation_text': 'تم نے غلط کیا.'}]

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")
tokenizer.src_lang = "en_XX"
tokenizer.tgt_lang = "ur_PK"


In [6]:
english_sentence = dataset["train"][1]["translation"]["en"]
urdu_sentence = dataset["train"][1]["translation"]["ur"]
print(english_sentence,"\n", urdu_sentence)

Except, among them, Your chosen servants." 
 بجز تیرے اُن بندوں کے جنہیں تو نے خالص کر لیا ہے"


In [7]:
inputs = tokenizer(english_sentence, text_target=urdu_sentence, return_tensors="pt")
inputs

{'input_ids': tensor([[2295,    2,  239,   34,    2,  735, 1898,  774,  191,    0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([[ 427, 3112,  634,   35,   40,  117,  628,    6,  612,   27,   18, 2510,
           47,  304,    8,  419,    0]])}

In [8]:
# decode the token back to the english and urdu
print(tokenizer.batch_decode(inputs.input_ids, skip_special_tokens=True))
print(tokenizer.batch_decode(inputs.labels, skip_special_tokens=True))

['Except, among them, Your chosen servants."']
['بجز تیرے اُن بندوں کے جنہیں تو نے خالص کر لیا ہے"']


In [9]:
wrong_targets = tokenizer(urdu_sentence)
print(tokenizer.convert_ids_to_tokens(wrong_targets["input_ids"]))
print(tokenizer.convert_ids_to_tokens(inputs["labels"][0]))

['▁', 'ب', 'ج', 'ز', '▁', 'ت', 'ی', 'ر', 'ے', '▁', 'ا', 'ُ', 'ن', '▁', 'ب', 'ن', 'د', 'و', 'ں', '▁', 'کے', '▁', 'ج', 'ن', 'ہ', 'ی', 'ں', '▁', 'ت', 'و', '▁', 'ن', 'ے', '▁', 'خ', 'ا', 'ل', 'ص', '▁', 'ک', 'ر', '▁', 'ل', 'ی', 'ا', '▁', 'ہے', '"', '</s>']
['▁ب', 'جز', '▁تیرے', '▁ا', 'ُ', 'ن', '▁بندوں', '▁کے', '▁جنہیں', '▁تو', '▁نے', '▁خالص', '▁کر', '▁لیا', '▁ہے', '"', '</s>']


In [10]:
max_length = 128
def preprocess_function(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["ur"] for ex in examples["translation"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs

In [11]:
preprocess_function(dataset["train"][1:2])

{'input_ids': [[2295, 2, 239, 34, 2, 735, 1898, 774, 191, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[427, 3112, 634, 35, 40, 117, 628, 6, 612, 27, 18, 2510, 47, 304, 8, 419, 0]]}

In [12]:
tokenized_datasets = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names,
)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/753913 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [13]:
tokenizer.decode(tokenized_datasets["train"][1]["labels"])

'بجز تیرے اُن بندوں کے جنہیں تو نے خالص کر لیا ہے"</s>'

## Fine-tuning the model with the Trainer API

In [14]:
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [15]:
text = "what's your name?"

tokenizer.src_lang = "en_XX"
inputs = tokenizer(text, return_tensors="pt")
generated_tokens = model.generate(
    **inputs,
)

print("English:", text)
print("Urdu:", tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0])


English: what's your name?
Urdu: آپ کا نام کیا ہے؟


In [16]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [17]:
import numpy as np
import evaluate

metric = evaluate.load("sacrebleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [21]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"MarianMT-english-urdu",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32, # Reduced the batch size
    per_device_eval_batch_size=32,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)


In [22]:
from transformers import Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

In [23]:
# we have to evaluate the model before the model  training
trainer.evaluate(max_length=max_length)
# trainer.push_to_hub()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: azheraly009 (azheraly009-nust) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


{'eval_loss': 0.8789143562316895,
 'eval_model_preparation_time': 0.0054,
 'eval_bleu': 25.063070808481694,
 'eval_runtime': 125.834,
 'eval_samples_per_second': 15.894,
 'eval_steps_per_second': 0.501}

In [24]:
trainer.train()

Step,Training Loss
500,0.213600
1000,0.213700
1500,0.214700
2000,0.214200
2500,0.216600
3000,0.206700
3500,0.212400
4000,0.205900
4500,0.209600
5000,0.200200


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3917: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62024]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Step,Training Loss
500,0.213600
1000,0.213700
1500,0.214700
2000,0.214200
2500,0.216600
3000,0.206700
3500,0.212400
4000,0.205900
4500,0.209600
5000,0.200200


TrainOutput(global_step=70680, training_loss=0.16568706725098417, metrics={'train_runtime': 14192.9436, 'train_samples_per_second': 159.357, 'train_steps_per_second': 4.98, 'total_flos': 5.618044584257126e+16, 'train_loss': 0.16568706725098417, 'epoch': 3.0})

In [25]:
trainer.evaluate(max_length=max_length)

{'eval_loss': 0.8389467597007751,
 'eval_model_preparation_time': 0.0054,
 'eval_bleu': 14.87102762289686,
 'eval_runtime': 142.9295,
 'eval_samples_per_second': 13.993,
 'eval_steps_per_second': 0.441,
 'epoch': 3.0}

In [26]:
trainer.push_to_hub()

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...nt/MarianMT-english-urdu/target.spm: 100%|##########|  848kB /  848kB            

  ...anMT-english-urdu/training_args.bin: 100%|##########| 5.91kB / 5.91kB            

  ...vents.1756280879.73447d6a19ec.789.0: 100%|##########| 36.7kB / 36.7kB            

  ...nt/MarianMT-english-urdu/source.spm: 100%|##########|  816kB /  816kB            

  ...anMT-english-urdu/model.safetensors:  11%|#1        | 33.6MB /  304MB            

  ...vents.1756296728.73447d6a19ec.789.1: 100%|##########|   480B /   480B            

CommitInfo(commit_url='https://huggingface.co/azherali/MarianMT-english-urdu/commit/473c82a39455e0fa8c2c06b62e5db57f787fdba8', commit_message='End of training', commit_description='', oid='473c82a39455e0fa8c2c06b62e5db57f787fdba8', pr_url=None, repo_url=RepoUrl('https://huggingface.co/azherali/MarianMT-english-urdu', endpoint='https://huggingface.co', repo_type='model', repo_id='azherali/MarianMT-english-urdu'), pr_revision=None, pr_num=None)

In [35]:
model_checkpoint = "azherali/MarianMT-english-urdu"
translator = pipeline("translation", model=model_checkpoint, src_lang="en_XX", tgt_lang="ur_PK")
translator("you did wrong.")

Device set to use cuda:0


[{'translation_text': 'تم نے غلط کیا.'}]

In [32]:
translator("what's your name?")

[{'translation_text': 'آپ کا نام کیا ہے؟'}]

In [38]:
# Your fine-tuned checkpoint
model_checkpoint = "azherali/MarianMT-english-urdu"

# Create translation pipeline
translator = pipeline(
    "translation",
    model=model_checkpoint,
    tokenizer=model_checkpoint,
    src_lang="en_XX",
    tgt_lang="ur_PK"
)
# Example translation
result = translator("I am very happy today.")
print(result[0]['translation_text'])



Device set to use cuda:0


میں آج بہت خوش ہوں.


میں آج بہت خوش ہوں.
